In [ ]:
# -------------------------------------------------------- (GUI + MongoDb ) ---------------------------------------------------------------------

import os
import pandas as pd
import tkinter as tk
from tkinter import scrolledtext
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi    

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()


# Retrieve the keys from environment variables
uri = os.getenv("MONGODB_URI")
openai_api_key = os.getenv("OPENAI_API_KEY")

print(f"URI , Api_KEY{uri,openai_api_key}")

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = openai_api_key

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'), maxPoolSize=1000)

# Ping the server to confirm the connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)



# Connect to the database and collection
db = client['test']  # Replace with your actual database name
collection = db['purchase_order']  # Replace with your actual collection name

#print("Quering the data from MongoDB...")
# Query the data from MongoDB
data = list(collection.find({}))  # Retrieves all documents from the collection
#print(f"DATA: {data}")
# Convert the MongoDB data to a pandas DataFrame
df = pd.DataFrame(data)

# Drop the MongoDB-specific `_id` field if it exists
if '_id' in df.columns:
    df = df.drop(columns=['_id'])

#print("Initializing Model...")

# Initialize the ChatOpenAI with GPT-4-turbo
llm = ChatOpenAI(temperature=0.5, model="gpt-4-turbo")

# Function to handle the prompt and display the result
def handle_prompt():
    prompt = prompt_entry.get()  # Get the user's prompt from the input field
    agent_executor = create_pandas_dataframe_agent(llm, df, allow_dangerous_code=True, verbose=True)
    result = agent_executor.invoke(prompt)  # Invoke the agent with the user's prompt
    print(f"Output from model: {result}")
    # Assuming the result is in a dictionary, extract the plain text
    if isinstance(result, dict):
        output = result.get("output", "No output found")  # Modify 'output' if necessary
    else:
        output = result
    
    # Display the output in the output text box
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, output)

# Create the main application window
root = tk.Tk()
root.title("LangChain MongoDB Agent")

# Create and pack the input prompt label and entry
prompt_label = tk.Label(root, text="Enter your prompt:")
prompt_label.pack(pady=5)

prompt_entry = tk.Entry(root, width=80)
prompt_entry.pack(pady=5)

# Create and pack the submit button
submit_button = tk.Button(root, text="Submit", command=handle_prompt)
submit_button.pack(pady=5)

# Create and pack the output text box (with a scrollbar)
output_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=100, height=20)
output_text.pack(pady=10)

# Run the GUI application
root.mainloop()

In [ ]:
# ------------------------   GUI TEST Without Mongo DB (Run this if mongodb one takes too long)      ---------------------------------------------------------------------------

import os
import pandas as pd
import tkinter as tk
from tkinter import scrolledtext
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()


# Retrieve the keys from environment variables
uri = os.getenv("MONGODB_URI")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Load the dataset
df = pd.read_csv("dataset/data.csv")

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = openai_api_key
# Initialize the ChatOpenAI with GPT-4-turbo
llm = ChatOpenAI(temperature=0.5, model="gpt-4-turbo")

# Create the CSV agent
agent_executor = create_csv_agent(llm, "dataset/data.csv", allow_dangerous_code=True, verbose=True)

# Function to handle the prompt and display the result
def handle_prompt():
    prompt = prompt_entry.get()  # Get the user's prompt from the input field
    result = agent_executor.invoke(prompt)  # Invoke the agent with the user's prompt
    
    # Assuming the result is in a dictionary, extract the plain text
    if isinstance(result, dict):
        output = result.get("output", "No output found")  # Modify 'output' if necessary
    else:
        output = result
    
    # Display the output in the output text box
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, output)

# Create the main application window
root = tk.Tk()
root.title("LangChain CSV Agent")

# Create and pack the input prompt label and entry
prompt_label = tk.Label(root, text="Enter your prompt:")
prompt_label.pack(pady=5)

prompt_entry = tk.Entry(root, width=80)
prompt_entry.pack(pady=5)

# Create and pack the submit button
submit_button = tk.Button(root, text="Submit", command=handle_prompt)
submit_button.pack(pady=5)

# Create and pack the output text box (with a scrollbar)
output_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=100, height=20)
output_text.pack(pady=10)

# Run the GUI application
root.mainloop()


In [ ]:
# --------------------------------------- Testing Paramters to verify model answer accuracy -----------------------------------

import pandas as pd

# Load the dataset
df = pd.read_csv("dataset/data.csv")

number_of_entries = df[df['Fiscal Year'] == '2014-2015'].shape[0]

# Display the result
print(f"Number of entries for the fiscal year 2014-2015: {number_of_entries}")

# Get the most frequently mentioned item
most_frequent_item = df['Item Name'].value_counts().idxmax()

# Get the count of that item
most_frequent_count = df['Item Name'].value_counts().max()

# Display the result
print(f"The most frequently mentioned item is: {most_frequent_item} with {most_frequent_count} mentions")


# Count the occurrences of each item
item_counts = df['Item Name'].value_counts()

# Get the least 3 ordered items
least_three_items = item_counts.nsmallest(3)

# Display the result
print("The three least frequently ordered items are:")
print(least_three_items)


# Group by 'Item Name' and sum the 'Quantity' to get total sales per item
item_sales = df.groupby('Item Name')['Quantity'].sum().reset_index()

# Find the most sold item
most_sold = item_sales.loc[item_sales['Quantity'].idxmax()]
print("Most Sold Item:")
print(most_sold)

# Find the least sold item
least_sold = item_sales.loc[item_sales['Quantity'].idxmin()]
print("Least Sold Item:")
print(least_sold)

In [ ]:
# ---------------------------------------------- (Single Query test) --------------------------------------------------------
import os
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi    
from bson import ObjectId  # Import to handle ObjectId type
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the MongoDB URI from the environment
uri = os.getenv("MONGODB_URI")

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Connect to the database and collection
db = client['test']  # Replace with your actual database name
collection = db['purchase_order']  # Replace with your actual collection name

# Example _id value (replace with your actual _id)
document_id = "66d4b7c3ac52713bdc01404b"  # This should be an ObjectId string

# Convert the _id string to ObjectId type if necessary
try:
    object_id = ObjectId(document_id)
    
    # Retrieve a single document by _id
    result = collection.find_one({"_id": object_id})
    
    # Print the result
    if result:
        print("Document found:", result)
    else:
        print("No document found with that _id.")
except Exception as e:
    print(f"Error occurred: {e}")
